# Dependencies


## Installation


In [1]:
%pip install pip --upgrade -q
%pip install pandas -q
%pip install numpy -q
%pip install matplotlib -q
%pip install folium -q
%pip install geopy -q
%pip install geopandas -q


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Importing


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import geopandas as gpd

# Loading Data


In [3]:
DTYPE = {
    "IBGE-code": str, 
    "IBGE-state-code": str, 
    "IBGE-municipality-code": str,
}

GEO_DATA_PATH = "./data/generated/geo/municipalities_geodata.geojson"
SUS_DATA = "./data/generated/sus/mental_diseases_jan2024-dec2024.csv"

In [4]:
from numpy import dtype

geo_data = gpd.read_file(GEO_DATA_PATH, encoding="utf-8")
sus_data = pd.read_csv(SUS_DATA, encoding="utf-8", sep=";", dtype=DTYPE)

In [5]:
# GeoJson dont accept dtype "object" in columns with numbers, so we need to convert them to str
geo_data["IBGE-code"] = geo_data["IBGE-code"].astype(str)
geo_data["IBGE-state-code"] = geo_data["IBGE-state-code"].astype(str)
geo_data["IBGE-municipality-code"] = geo_data["IBGE-municipality-code"].astype(str)

In [6]:
# Display the first few rows of the GeoDataFrame
geo_data.head(5)

,IBGE-code,IBGE-state-code,IBGE-municipality-code,municipality-state,municipality-name,latitude,longitude,geometry
0,110001,11,0001,Rondônia,Alta Floresta D'Oeste,-62.27466099683791,-12.470132272726424,"POLYGON ((-61.93075 -11.97349, -61.9332 -11.97..."
1,110002,11,0002,Rondônia,Ariquemes,-62.95724463217704,-9.95188002813111,"POLYGON ((-63.17933 -10.13924, -63.17237 -10.1..."
2,110003,11,0003,Rondônia,Cabixi,-60.63980722186971,-13.474878560170035,"POLYGON ((-60.56035 -13.32206, -60.37162 -13.3..."
3,110004,11,0004,Rondônia,Cacoal,-61.32472873458685,-11.301287369709485,"POLYGON ((-61.00075 -11.25298, -61.00103 -11.3..."
4,110005,11,0005,Rondônia,Cerejeiras,-61.26095018379903,-13.203561520699402,"POLYGON ((-60.82068 -13.11944, -60.81773 -13.1..."


In [7]:
# Display the first few rows of the Sus_data DataFrame
sus_data.head(5)

,IBGE-code,IBGE-state-code,IBGE-municipality-code,internacoes
0,110001,11,0001,54
1,110002,11,0002,14
2,110004,11,0004,48
3,110006,11,0006,2
4,110009,11,0009,1


# Joining Data


In [8]:
# Joining the SUS data with the geographical data to get the municipalities with reports
municipalities_with_reports = geo_data.merge(sus_data, how="inner", on=["IBGE-code", "IBGE-state-code", "IBGE-municipality-code"])
municipalities_with_reports = gpd.GeoDataFrame(
    municipalities_with_reports[["IBGE-code", "IBGE-state-code", "IBGE-municipality-code", "municipality-name", "municipality-state", "internacoes"]],
    geometry=municipalities_with_reports["geometry"],
)
municipalities_with_reports.head(5)

,IBGE-code,IBGE-state-code,IBGE-municipality-code,municipality-name,municipality-state,internacoes,geometry
0,110001,11,0001,Alta Floresta D'Oeste,Rondônia,54,"POLYGON ((-61.93075 -11.97349, -61.9332 -11.97..."
1,110002,11,0002,Ariquemes,Rondônia,14,"POLYGON ((-63.17933 -10.13924, -63.17237 -10.1..."
2,110004,11,0004,Cacoal,Rondônia,48,"POLYGON ((-61.00075 -11.25298, -61.00103 -11.3..."
3,110006,11,0006,Colorado do Oeste,Rondônia,2,"POLYGON ((-60.41495 -13.04076, -60.41593 -13.0..."
4,110009,11,0009,Espigão D'Oeste,Rondônia,1,"POLYGON ((-61.07244 -11.63551, -61.21172 -11.5..."


In [9]:
# Joining the SUS data with the geographical data to get the municipalities without reports
municipalities_without_reports = geo_data.copy()

municipalities_without_reports = municipalities_without_reports.merge(sus_data, how="left", on=["IBGE-code", "IBGE-state-code", "IBGE-municipality-code"])
municipalities_without_reports = municipalities_without_reports[municipalities_without_reports["internacoes"].isna()]
municipalities_without_reports = municipalities_without_reports.drop(columns=["internacoes"])
municipalities_without_reports.head(5)

,IBGE-code,IBGE-state-code,IBGE-municipality-code,municipality-state,municipality-name,latitude,longitude,geometry
2,110003,11,0003,Rondônia,Cabixi,-60.63980722186971,-13.474878560170035,"POLYGON ((-60.56035 -13.32206, -60.37162 -13.3..."
4,110005,11,0005,Rondônia,Cerejeiras,-61.26095018379903,-13.203561520699402,"POLYGON ((-60.82068 -13.11944, -60.81773 -13.1..."
6,110007,11,0007,Rondônia,Corumbiara,-61.09164645391249,-12.926629422648453,"POLYGON ((-61.00051 -13.1052, -61.01748 -13.11..."
7,110008,11,0008,Rondônia,Costa Marques,-64.05877173931918,-12.147250102409899,"POLYGON ((-64.14378 -12.48169, -64.14578 -12.4..."
23,110033,11,0033,Rondônia,Nova Mamoré,-64.62909951864042,-10.382167455612384,"POLYGON ((-64.21185 -10.76905, -64.21829 -10.7..."


In [10]:
# Saving the dataframes
MUNICIPALITIES_WITH_REPORTS_FILE_PATH = "data/generated/mixed/municipalities_with_reports_jan2024-dec2024.geojson"
MUNICIPALITIES_WITHOUT_REPORTS_FILE_PATH = "data/generated/mixed/municipalities_without_reports_jan2024-dec2024.geojson"
# Saving the dataframes as GeoJSON files
municipalities_with_reports.to_file(MUNICIPALITIES_WITH_REPORTS_FILE_PATH, driver="GeoJSON", encoding="utf-8")
municipalities_without_reports.to_file(MUNICIPALITIES_WITHOUT_REPORTS_FILE_PATH, driver="GeoJSON", encoding="utf-8")


# Generating Map


In [11]:
BRAZILIA_CORDINATES = [-15.7801, -47.9292]
m = folium.Map(location=BRAZILIA_CORDINATES, zoom_start=4, tiles='cartodb positron', embed=False,)
fg = folium.FeatureGroup(name="Municipalities with and whitout reports", show=True)

In [12]:
# Municipalities without reports
folium.GeoJson(
    municipalities_without_reports[["IBGE-code", "IBGE-state-code", "IBGE-municipality-code", "municipality-state", "municipality-name", "geometry"]],
    name="Municipalities without reports",
    style_function=lambda x: {
        "color": "black",
        "weight": 1,
        "fillOpacity": 1,
        "fillColor": "red",
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["IBGE-code", "IBGE-state-code", "IBGE-municipality-code", "municipality-state", "municipality-name"],
        aliases=["IBGE code", "IBGE state code", "IBGE municipality code", "Municipality state", "Municipality name"],
        localize=True,
        sticky=False,
        labels=True,
        style="background-color: white; color: black; font-family: Arial; font-size: 12px; padding: 5px;",
    ),
).add_to(fg)

In [13]:
# Municipalities with reports
folium.GeoJson(
    municipalities_with_reports[["IBGE-code", "IBGE-state-code", "IBGE-municipality-code", "municipality-state", "municipality-name", "internacoes", "geometry"]],
    name="Municipalities with reports",
    style_function=lambda x: {
        "color": "black",
        "weight": 1,
        "fillOpacity": 1,
        "fillColor": "green",
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["IBGE-code", "IBGE-state-code", "IBGE-municipality-code", "municipality-state", "municipality-name", "internacoes"],
        aliases=["IBGE code", "IBGE state code", "IBGE municipality code", "Municipality state", "Municipality name" , "Internacoes"],
        localize=True,
        sticky=False,
        labels=True,
        style="background-color: white; color: black; font-family: Arial; font-size: 12px; padding: 5px;",
    ),
).add_to(fg)
fg.add_to(m)

In [14]:
# Render the map
# m

In [15]:
MUNICIPALITIES_WITH_AND_WITHOUT_REPORTS_MAP_PATH = "maps/municipalities_with_and_without_reports_jan2024-dec2024.html"
m.save(MUNICIPALITIES_WITH_AND_WITHOUT_REPORTS_MAP_PATH)

## Generating Choropleth


In [16]:
# This is for removing complexity from the map, so we can see the municipalities with reports info and the  map file'll be smaller
municipalities_with_reports_simplified = gpd.GeoDataFrame(
    municipalities_with_reports[["IBGE-code", "IBGE-state-code", "IBGE-municipality-code", "municipality-state", "municipality-name", "internacoes"]],
    geometry=municipalities_with_reports["geometry"].copy().simplify(tolerance=0.1, preserve_topology=True)
)

In [17]:
m = folium.Map(location=BRAZILIA_CORDINATES, zoom_start=4, tiles='cartodb positron', embed=False,)

In [18]:
# Folium GeoJson for municipalities without reports
folium.GeoJson(
    municipalities_without_reports[["IBGE-code", "IBGE-state-code", "IBGE-municipality-code", "municipality-state", "municipality-name", "geometry"]],
    name="Municipalities without reports",
    style_function=lambda x: {
        "color": "gray",
        "weight": 1,
        "fillColor": "black",
        "fillOpacity": 1,
        "line_color": "white",
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["IBGE-code", "IBGE-state-code", "IBGE-municipality-code", "municipality-state", "municipality-name"],
        aliases=["IBGE code", "IBGE state code", "IBGE municipality code", "Municipality state", "Municipality name"],
        localize=True,
        sticky=False,
        labels=True,
        style="background-color: white; color: black; font-family: Arial; font-size: 12px; padding: 5px;",
    ),
).add_to(m)

In [19]:
# Folium Choropleth map for municipalities with reports
folium.Choropleth(
    name ="Municipalities with reports - Total Hospitalizations",
    geo_data=municipalities_with_reports[["IBGE-code", "geometry"]],
    data=municipalities_with_reports[["IBGE-code", "internacoes"]],
    columns=["IBGE-code", "internacoes"],
    key_on="feature.properties.IBGE-code",
    fill_color="YlGn",
    line_opacity=0,
    legend_name="Hospitalizations",
).add_to(m)

In [20]:
# Generating a transparent overlay for the municipalities with reports info
folium.GeoJson(
    municipalities_with_reports_simplified,
    name="Municipalities with reports overlay",
    style_function=lambda x: {
        "color": "black",
        "weight": 0,
        "fillOpacity": 0,
        "fillColor": "white",
        "line_opacity": 0,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["IBGE-code", "IBGE-state-code", "IBGE-municipality-code", "municipality-state", "municipality-name", "internacoes"],
        aliases=["IBGE code", "IBGE state code", "IBGE municipality code", "Municipality state", "Municipality name" , "Internacoes"],
        localize=True,
        sticky=False,
        labels=True,
        style="background-color: white; color: black; font-family: Arial; font-size: 12px; padding: 5px;",
    ),
).add_to(m)
folium.LayerControl().add_to(m)

In [21]:
# Render the map
# m

In [22]:
# Saving the Folium map to an HTML file
CHOROPLETH_ABSOLUTE_CASES = "maps/choropleth_absolute_cases_jan2024-dec2024.html"
m.save(CHOROPLETH_ABSOLUTE_CASES)

# Testing


In [23]:
# Display the number of municipalities with reports and without reports
print(f"Municipalities with reports: {len(municipalities_with_reports)}")
print(f"Municipalities without reports: {len(municipalities_without_reports)}")
# Display the total number of municipalities in Brazil
print(f"Total municipalities in Brazil: {len(geo_data)}")
print(f"Total municipalities with reports: {len(municipalities_with_reports)}")
print(f"Total municipalities without reports: {len(municipalities_without_reports)}")


Municipalities with reports: 1904
Municipalities without reports: 3668
Total municipalities in Brazil: 5572
Total municipalities with reports: 1904
Total municipalities without reports: 3668
